In [39]:
import pandas as pd
from keras.optimizers import SGD
from src.learner import *
import time
import random

Some tuning for the `kidwords` set of words.

In [4]:
# inputs and outputs
X = np.genfromtxt('data/kidwords/orth-kid.csv', delimiter=",")
Y = np.genfromtxt('data/kidwords/phon-kid.csv', delimiter=",")
words = pd.read_csv('data/kidwords/kidwords.csv', header=None)[0].tolist()

For tuning we will use a random sample of the same size that our samples will eventually be. This involves allocating 600 words for test, and the rest for train - but not using our pre-allocated samples for the purpose. This is the same procedure as other tuning efforts here.

In [5]:
np.random.seed(982)

target_train_size = 300

train_n = X.shape[0]
test_n = train_n-target_train_size

sample = np.full(train_n, True, dtype=bool)

indices = np.random.choice(train_n, test_n, replace=False)

# Set chosen indices to True because they select the test items not the train items
sample[indices] = False

Search across HPs, all using SGD...

In [23]:
seed = 387
random.seed(seed)

In [25]:
with open('outputs/tune_kidwords_2_sgd.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                            "hidden_units",
                                            "learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.01, .025, .05, .075, .1, .15, .2, .25]: 
        for batch_size in [16, 32, 64, 96, 128, 256]:
            for epochs in [400, 450, 500, 550, 600]:
                for hidden in [100]:

                    optimizer = SGD(learning_rate=learning_rate)

                    model = learner(X, Y, seed, hidden, optimizer=optimizer)
                    
                    start_time = time.time()

                    model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

                    end_time = time.time()
                    runtime = end_time - start_time

                    loss_train, accuracy_train, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) 
                    loss_test, accuracy_test, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) 

                    f.write("{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    learning_rate,
                                                    batch_size,
                                                    epochs,
                                                    loss_train,
                                                    accuracy_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    mse_test,
                                                    runtime))
f.close()

Let's look at a specific set of hyperparameters for reference:

In [40]:
hidden = 100
learning_rate = 0.25

In [99]:
optimizer = SGD(learning_rate=learning_rate)
model = learner(X, Y, seed, hidden, optimizer=optimizer)                    
model.fit(X[sample], Y[sample], epochs=140, batch_size=256, verbose=True)

Epoch 1/140
2/2 [==============================] - 0s 5ms/step - loss: 0.6968 - binary_accuracy: 0.4950 - mse: 0.2518
Epoch 2/140
2/2 [==============================] - 0s 2ms/step - loss: 0.6955 - binary_accuracy: 0.5009 - mse: 0.2511
Epoch 3/140
2/2 [==============================] - 0s 4ms/step - loss: 0.6941 - binary_accuracy: 0.5082 - mse: 0.2505
Epoch 4/140
2/2 [==============================] - 0s 3ms/step - loss: 0.6928 - binary_accuracy: 0.5151 - mse: 0.2498
Epoch 5/140
2/2 [==============================] - 0s 3ms/step - loss: 0.6915 - binary_accuracy: 0.5216 - mse: 0.2492
Epoch 6/140
2/2 [==============================] - 0s 4ms/step - loss: 0.6902 - binary_accuracy: 0.5311 - mse: 0.2485
Epoch 7/140
2/2 [==============================] - 0s 8ms/step - loss: 0.6889 - binary_accuracy: 0.5398 - mse: 0.2479
Epoch 8/140
2/2 [==============================] - 0s 5ms/step - loss: 0.6876 - binary_accuracy: 0.5451 - mse: 0.2472
Epoch 9/140
2/2 [==============================] - 0s 4m

Iterate through random samples and train/ save in longruns...

In [100]:
model.fit(X[sample], Y[sample], epochs=70, batch_size=1, verbose=True)

Epoch 1/70
300/300 [==============================] - 0s 1ms/step - loss: 0.1805 - binary_accuracy: 0.9460 - mse: 0.0449
Epoch 2/70
300/300 [==============================] - 0s 964us/step - loss: 0.1376 - binary_accuracy: 0.9466 - mse: 0.0390
Epoch 3/70
300/300 [==============================] - 0s 847us/step - loss: 0.1292 - binary_accuracy: 0.9469 - mse: 0.0380
Epoch 4/70
300/300 [==============================] - 0s 965us/step - loss: 0.1248 - binary_accuracy: 0.9478 - mse: 0.0373
Epoch 5/70
300/300 [==============================] - 0s 1ms/step - loss: 0.1212 - binary_accuracy: 0.9482 - mse: 0.0364
Epoch 6/70
300/300 [==============================] - 0s 1ms/step - loss: 0.1182 - binary_accuracy: 0.9492 - mse: 0.0355
Epoch 7/70
300/300 [==============================] - 0s 1ms/step - loss: 0.1150 - binary_accuracy: 0.9510 - mse: 0.0346
Epoch 8/70
300/300 [==============================] - 0s 999us/step - loss: 0.1118 - binary_accuracy: 0.9519 - mse: 0.0335
Epoch 9/70
300/300 [====

In [103]:
hidden = 100
learning_rate = 0.25
epochs = 210


for s in range(seed, seed+20): # manipulate random seed by one starting with seed from above 

    random.seed(s)
    target_train_size = 300

    train_n = X.shape[0]
    test_n = train_n-target_train_size

    sample = np.full(train_n, True, dtype=bool)

    indices = np.random.choice(train_n, test_n, replace=False)

    # Set chosen indices to True because they select the test items not the train items
    sample[indices] = False

    train_indices = np.where(sample)[0]
    test_indices = np.where(~sample)[0]    

    split = []

    for i, word in enumerate(words):
        if i in train_indices:
            split.append('train')
        elif i in test_indices:
            split.append('test')

    model = learner(X, Y, seed=s, hidden=hidden, optimizer=SGD(learning_rate=learning_rate))

    # accuracy at epoch 0
    epoch = 0
    preds = model.predict(X, batch_size=batch_size) 
    preds_binary = (preds > 0.5).astype(int)
    accuracies = (preds_binary == Y).astype(int)

    fname = 'outputs/kidwords_2_sgd_longitudinal/tune_kidwords_2_sgd_longitudinal_seed_' + str(s) + '_epoch_' + str(epoch) + '.csv'

    df = pd.DataFrame(accuracies.tolist())
    df.columns = [colname + 1 for colname in df.columns]
    df['words'] = words
    df['train_test'] = split
    df.to_csv(fname, index=False)

    for epoch in range(epochs):

        if epoch < 140:
            batch_size = 256
        else:
            batch_size = 1
        
        fname = 'outputs/kidwords_2_sgd_longitudinal/tune_kidwords_2_sgd_longitudinal_seed_' + str(s) + '_epoch_' + str(epoch) + '.csv'

        model.fit(X[sample], Y[sample], epochs=1, batch_size=batch_size, verbose=True)

        preds = model.predict(X, batch_size=batch_size) 
        preds_binary = (preds > 0.5).astype(int)
        accuracies = (preds_binary == Y).astype(int)

        df = pd.DataFrame(accuracies.tolist())
        df.columns = [colname + 1 for colname in df.columns]
        df['words'] = words
        df['train_test'] = split
        df.to_csv(fname, index=False)

f.close()

2869/2869 [==============================] - 1s 333us/step


In [33]:
from keras.optimizers import SGD
model = learner(X, Y, seed=s, hidden=hidden, optimizer=SGD())

model.fit(X[sample], Y[sample], epochs=400, batch_size=batch_size, verbose=True)

Epoch 1/400
19/19 [==============================] - 0s 1ms/step - loss: 0.6961 - binary_accuracy: 0.4967 - mse: 0.2515
Epoch 2/400
19/19 [==============================] - 0s 928us/step - loss: 0.6956 - binary_accuracy: 0.4997 - mse: 0.2512
Epoch 3/400
19/19 [==============================] - 0s 974us/step - loss: 0.6950 - binary_accuracy: 0.5031 - mse: 0.2509
Epoch 4/400
19/19 [==============================] - 0s 995us/step - loss: 0.6945 - binary_accuracy: 0.5064 - mse: 0.2507
Epoch 5/400
19/19 [==============================] - 0s 1ms/step - loss: 0.6940 - binary_accuracy: 0.5099 - mse: 0.2504
Epoch 6/400
19/19 [==============================] - 0s 877us/step - loss: 0.6934 - binary_accuracy: 0.5130 - mse: 0.2501
Epoch 7/400
19/19 [==============================] - 0s 954us/step - loss: 0.6929 - binary_accuracy: 0.5161 - mse: 0.2499
Epoch 8/400
19/19 [==============================] - 0s 936us/step - loss: 0.6924 - binary_accuracy: 0.5191 - mse: 0.2496
Epoch 9/400
19/19 [=========

In [ ]:
df

In [ ]:
accuracy = batch_accuracy(Y, model.predict(X), dichotomous=False)

In [ ]:
sum([1 for acc in accuracy if acc == True])

In [ ]:
preds = (model.predict(X) > .5).astype(int)

In [ ]:
np.savetxt('outputs/tune_3k_1a_prediction_accuracies_sample_run.csv', (preds == Y).astype(int), delimiter = ',')

Now let's take this configuration and run over a set of random seeds and generate some longitudinal data to examine. 